## ETL_ MOVIES
TRANSFORMACIONES: 
- Algunos campos, como **belongs_to_collection**, **production_companies** y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.<P>

- Los valores nulos de los campos **revenue**, **budget** deben ser rellenados por el número 0.<P>

- Los valores nulos del campo **release date** deben eliminarse.<P>
 
- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna **release_year** donde extraerán el año de la fecha de estreno. <P>

- Crear la columna con el retorno de inversión, llamada **return** con los campos **revenue** y **budget**, dividiendo estas dos últimas **revenue** / **budget**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0. <P>

- Eliminar las columnas que no serán utilizadas, **video**,**imdb_id**,**adult**,**original_title**,**poster_path** y **homepage**. <P>

In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import ast
import sys
import os

# Agrega el directorio superior al path
sys.path.append(os.path.abspath(".."))

from funciones.desanidar_concatena import desanidar_concat
from funciones.reordena_col import reordena_colum


### 0. Carga de los datos

In [2]:
movie_test  = pd.read_csv('../datasets_original/movies_dataset.csv', sep=',',quotechar='"', encoding='utf-8' )

C:\Users\Angelica\AppData\Local\Temp\ipykernel_24104\63210814.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_test  = pd.read_csv('../datasets_original/movies_dataset.csv', sep=',',quotechar='"', encoding='utf-8' )


In [3]:
print(movie_test.shape)

(45466, 24)


Copia del dataset para aplicar las transformaciones

In [4]:
movie_test_df = movie_test.copy()

In [5]:
movie_test_df.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


### 1. TRANSFORMACION - Eliminación columnas

**Desanidamiento columnas**

In [6]:
for columna in ['belongs_to_collection','genres', 'production_companies', 'production_countries', 'spoken_languages']:
   movie_test_df = desanidar_concat(movie_test_df, columna)

In [7]:
print(len(movie_test.columns))
print(len(movie_test_df.columns))

24
31


**VALORES NULOS A '0'**
COLUMNAS:
- budget
- revenue
  
Conversión a enteros y valores nulos seteados con '0'
- pd.to_numeric   : convierte los valores de la columna, maneja errores 'coerce' (lo paso a nan)
- fillna(0)       : valores nan a 0
- astype('Int64') : valores enteros

In [8]:
movie_test_df["budget"]= pd.to_numeric(movie_test_df["budget"], errors='coerce').fillna(0).astype('Int64')
movie_test_df["revenue"]= pd.to_numeric(movie_test_df["revenue"], errors= 'coerce').fillna(0).astype('Int64')

In [9]:
#Se verifica cambios
print(type(movie_test["budget"][0]))
print(type(movie_test_df["budget"][0]))

print(type(movie_test["revenue"][0]))
print(type(movie_test_df["revenue"][0]))

<class 'str'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>


**COLUMNA RETURN**

return = revenue/budget (recaudacion/presupuesto)

In [10]:
movie_test_df["return"] = movie_test_df.apply(
                        lambda fila: round( fila['revenue']/fila['budget'], 2) if fila['budget']!= 0  else 0, axis = 1)

* ELIMINACIÓN VALORES NULOS 'release date' <p>
* COLUMNA RELEASE_YEAR

- Pasar de string a formato datetime (AAAA-mm-dd)
- Obtiene la columna release_year, a formato Int
- Se elimina las filas donde haya valores nulos en 'release_date'

In [11]:
movie_test_df["release_date"] = pd.to_datetime(movie_test_df["release_date"], errors= 'coerce', format='%Y-%m-%d')
movie_test_df["release_year"] = movie_test_df["release_date"].dt.year.astype('Int64')

movie_test_df = movie_test_df.dropna(subset= 'release_date')

movie_test_df[['release_date','release_year']].head(5)

,release_date,release_year
0,1995-10-30,1995
1,1995-12-15,1995
2,1995-12-22,1995
3,1995-12-22,1995
4,1995-02-10,1995


Comprobacion de eliminación de valores nulos

In [12]:
print(movie_test["release_date"].isnull().sum())
print(movie_test_df["release_date"].isnull().sum())
print(movie_test_df["release_year"].isnull().sum())

87
0
0


COLUMNAS a eliminar:

Se analiza valores nulos en cada columna

In [13]:
nulos = movie_test_df.isnull().sum()
print(nulos )

adult                                      0
budget                                     0
homepage                               37610
id                                         0
imdb_id                                   14
original_language                         11
original_title                             0
overview                                 941
popularity                                 0
poster_path                              339
release_date                               0
revenue                                    0
runtime                                  246
status                                    80
tagline                                24978
title                                      0
video                                      0
vote_average                               0
vote_count                                 0
id_belongs_to_collection               40889
name_belongs_to_collection             40889
poster_path_belongs_to_collection      41432
backdrop_p

- Columnas a elimniar_ video, imdb_id, adult, original_title, poster_path y homepage.
- Columna status, se va a eliminar despues de filtrar las peliculas released
- Columnas que no se van a usar en analisis o funciones: belongs_to_collection, 

In [14]:
movie_test_df[movie_test_df['status']=='released']

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,id_genres,name_genres,name_production_companies,id_production_companies,iso_3166_1_production_countries,name_production_countries,iso_639_1_spoken_languages,name_spoken_languages,return,release_year


In [15]:
columnas = [ 'video', 'imdb_id', 'adult', 'original_title', 'poster_path' , 'homepage','status' , 'tagline', 'id_belongs_to_collection', 
            'name_belongs_to_collection',
       'poster_path_belongs_to_collection',
       'backdrop_path_belongs_to_collection', 'name_production_companies', 'id_production_companies', 'name_production_countries',
       'iso_3166_1_production_countries','iso_639_1_spoken_languages', 'name_spoken_languages']


movie_test_df = movie_test_df.drop(columnas, axis=1)

In [16]:
movie_test_df.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,id_genres,name_genres,return,release_year
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,16,Animation,12.45,1995


**VALORES NULOS**

In [17]:
nulos = movie_test_df.isnull().sum()
print(nulos )

budget                 0
id                     0
original_language     11
overview             941
popularity             0
release_date           0
revenue                0
runtime              246
title                  0
vote_average           0
vote_count             0
id_genres              0
name_genres            0
return                 0
release_year           0
dtype: int64


In [18]:
# Eliminacion de filas por valores nulos 
movie_test_df = movie_test_df.dropna(subset=['overview','original_language' ,'runtime'])

Comparación dataset original vs ETL(dataset)

In [19]:
print(movie_test.shape)
print(movie_test_df.shape)

(45466, 24)
(44425, 15)


### CAMBIAR NOMBRE y TIPO DATOS (string => int, float, etc)

In [20]:
columnas = movie_test_df.columns.tolist()
for i in columnas:
    print(f'Tipo de dato de {i}:            { type(movie_test_df[i][0]) }')

Tipo de dato de budget:            <class 'numpy.int64'>
Tipo de dato de id:            <class 'str'>
Tipo de dato de original_language:            <class 'str'>
Tipo de dato de overview:            <class 'str'>
Tipo de dato de popularity:            <class 'float'>
Tipo de dato de release_date:            <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Tipo de dato de revenue:            <class 'numpy.int64'>
Tipo de dato de runtime:            <class 'numpy.float64'>
Tipo de dato de title:            <class 'str'>
Tipo de dato de vote_average:            <class 'numpy.float64'>
Tipo de dato de vote_count:            <class 'numpy.float64'>
Tipo de dato de id_genres:            <class 'numpy.int64'>
Tipo de dato de name_genres:            <class 'str'>
Tipo de dato de return:            <class 'numpy.float64'>
Tipo de dato de release_year:            <class 'numpy.int64'>


Cambiar tipo de datos
- id           (str => int)
- runtime      (float => int)
- vote_count   (float => int)
- popularity  a 2 decimales

In [21]:
movie_test_df["id"] = movie_test_df["id"].astype('Int64')
movie_test_df["runtime"] = movie_test_df["runtime"].astype('Int64')
movie_test_df["vote_count"] = movie_test_df["vote_count"].astype('Int64')

movie_test_df["popularity"] = movie_test_df["popularity"].astype(float)
movie_test_df["popularity"] = movie_test_df["popularity"].round(2)

In [22]:
movie_test_df.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,id_genres,name_genres,return,release_year
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.95,1995-10-30,373554033,81,Toy Story,7.7,5415,16,Animation,12.45,1995


Cambiar nombres a columnas:

In [23]:
movie_test_df  =movie_test_df.rename( {'id':'id_movie'} , axis=1)

### VERIFICACIÓN IDENTIFICADORES UNICOS

Cantidad de valores duplicados, luego de aparcer por segunda vez <p>
Identificador único: 'id'

In [24]:
duplicados= movie_test_df[movie_test_df["id_movie"].duplicated()]
print(f'Hay {len(duplicados)} id_movies duplicados')


Hay 30 id_movies duplicados


Eliminación de valores duplicados de la columna 'id_movie'

In [25]:
# keep='first' se mantiene el id original, y se elimina el duplicado
movie_test_df.drop_duplicates(subset='id_movie', keep='first', inplace=True)

Se verifica que ya no existen valores duplicados

In [26]:
duplicados2= movie_test_df[movie_test_df["id_movie"].duplicated()]
print(len(duplicados2))

0


Se genera tabla externa generos

In [ ]:
genres = movie_test_df[['id_genres', 'name_genres']].drop_duplicates().reset_index(drop=True)
movie_test_df = movie_test_df.drop(columns=['name_genres'])
genres

In [28]:
# Cambio del orden 'id_movie'
movie_test_df= reordena_colum(movie_test_df, 'id_movie', 0)

In [31]:
movie_test_df.head(1)

,id_movie,budget,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,id_genres,return,release_year
0,862,30000000,en,"Led by Woody, Andy's toys live happily in his ...",21.95,1995-10-30,373554033,81,Toy Story,7.7,5415,16,12.45,1995


### EXPORTACIÓN DATASET FINAL 

In [32]:
print(movie_test_df.shape)
movie_test_df.info()

(44395, 14)
<class 'pandas.core.frame.DataFrame'>
Index: 44395 entries, 0 to 45465
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_movie           44395 non-null  Int64         
 1   budget             44395 non-null  Int64         
 2   original_language  44395 non-null  object        
 3   overview           44395 non-null  object        
 4   popularity         44395 non-null  float64       
 5   release_date       44395 non-null  datetime64[ns]
 6   revenue            44395 non-null  Int64         
 7   runtime            44395 non-null  Int64         
 8   title              44395 non-null  object        
 9   vote_average       44395 non-null  float64       
 10  vote_count         44395 non-null  Int64         
 11  id_genres          44395 non-null  int64         
 12  return             44395 non-null  float64       
 13  release_year       44395 non-null  Int64         
dtyp

In [33]:
movie_test_df.to_parquet('../datasets/movie_dataset_final.parquet', engine='pyarrow' ,compression='snappy', index=False)
genres.to_parquet('../datasets/generos_df.parquet', engine='pyarrow' ,compression='snappy', index=False)